In [1]:
import importlib
import full_model

In [2]:
full_model.eng = full_model.start_matlab()


Starting MATLAB engine...
-------------------------
Elapsed: 3.06 sec


In [3]:
images_norm = full_model.loadmat("./saved/yaleB.mat")['X']
labels = full_model.loadmat("./saved/yaleB.mat")['Y'].reshape(-1)

## Try various Hyperparameter tuning methods
* [Scikit-Optimize](https://scikit-optimize.github.io/notebooks/hyperparameter-optimization.html)

In [4]:
importlib.reload(full_model)

<module 'full_model' from 'C:\\Users\\aleks_000\\Desktop\\Mentorship\\!GitHub\\full_model.py'>

In [5]:
from skopt.space import Real, Integer

space  = [Real(10**-2, 10**0, "log-uniform", name='lr_pretrain'),
          Real(10**-3, 10**-1, "log-uniform", name='lr'),
          Real(10**0, 10**2, "log-uniform", name='alpha1'),
          Integer(2, 32, name='maxIter1'),
          Real(10**0, 10**2, "log-uniform", name='alpha2'),
          Integer(2, 32, name='maxIter2'),]

In [35]:
from skopt.utils import use_named_args

@use_named_args(space)
def objective(**params):
    global ep_prtr_, ep_, seed_
    try:
        return 1-full_model.run_model(images_norm, labels, seed_, ep_prtr_, ep_, **params)[0]
        #return params['lr_pretrain']*params['lr']*params['alpha1']*params['alpha2']
    except Exception as ex:
        if(type(ex) == KeyError):
            raise ex
        return 1

In [36]:
from skopt.plots import plot_convergence
from skopt import dummy_minimize
import time

def optimize(function, iterations, epochs_pretrain, epochs, random_seed):
    start_time = time.time()
    
    # global b/c I couldn't find a better way to directly pass these to the objective function
    global ep_prtr_, ep_, seed_
    ep_prtr_ = epochs_pretrain
    ep_ = epochs
    seed_ = random_seed
    
    # kwargs b/c dummy_minimize can not take n_jobs
    kwargs = {"n_calls":iterations, "random_state":random_seed, "verbose":True}
    if(function != dummy_minimize):
        kwargs["n_jobs"] = -1
    res = function(objective, space, **kwargs)

    plot_convergence(res, yscale="log")
    print("\n------------------")
    print("Best score: {0:.4f}".format(res.fun))
    for i in range(10, iterations+1, 10):
        print("\t{0:d}: {1:.4f}".format(i, min(res.func_vals[0:i])))
    print("Best parameters: {0:}".format(res.x))
    print("Total time elapsed: {0:.2f} sec\n".format(time.time()-start_time))
    return res

In [8]:
def asdf(result):
    for m in result.models:
        print(id(m))

In [9]:
def helper(result):
    global models_, state_, set_vars_
    if(set_vars):
        result.models[:-1] = models_[:-1]
        result.models[-1] = models_[-1]
        result.random_state.set_state(state_)
        set_vars_ = False
    #asdf(result)

In [10]:
def reload(result, addtl_calls):
    args = result.specs['args']
    args['n_calls'] = addtl_calls
    args['n_random_starts'] = 0
    args['x0'] = result.x_iters
    args['y0'] = result.func_vals
    args['random_state'] = None
    args['callback'] = helper
    global models_, state_, set_vars_
    models_ = result.models
    state_ = result.random_state.get_state()
    set_vars_ = True
    return gp_minimize(**args)

In [ ]:
res1 = optimize(gp_minimize, 12, 1, 1, 0)

In [ ]:
res12 = reload(res1, 4)

In [ ]:
res2 = optimize(gp_minimize, 16, 1, 1, 0)

In [ ]:
# Pro-tip:
# Program taking too long?
# No problem
# ----------
# For a lossless termination, delete the temp folder,
# wait for the code to finish, then create it again
# or
# Deny Admin permissions to temp.mat,
# wait for the code to finish, then remove the restrictions
seed = 2
iters = 50
ep_prtr = 201
ep = 101

In [ ]:
from skopt import gp_minimize
gp = optimize(gp_minimize, iters, ep_prtr, ep, seed)

In [ ]:
from skopt import dummy_minimize
dummy = optimize(dummy_minimize, iters, ep_prtr, ep, seed)

In [ ]:
from skopt import forest_minimize
forest = optimize(forest_minimize, iters, ep_prtr, ep, seed)

In [ ]:
from skopt import gbrt_minimize
gbrt = optimize(gbrt_minimize, iters, ep_prtr, ep, seed)

In [32]:
from skopt import dump
dump(gp, "optims/scenario0/gp_" + str(seed) + "_" + str(iters) + ".opt")
dump(dummy, "optims/scenario0/dummy_" + str(seed) + "_" + str(iters) + ".opt")
dump(forest, "optims/scenario0/forest_" + str(seed) + "_" + str(iters) + ".opt")
dump(gbrt, "optims/scenario0/gbrt_" + str(seed) + "_" + str(iters) + ".opt")

In [28]:
from skopt import load
gp_loaded = load("optims/scenario0/gp_" + str(seed) + "_" + str(iters) + ".opt")
dummy_loaded = load("optims/scenario0/dummy_" + str(seed) + "_" + str(iters) + ".opt")
forest_loaded = load("optims/scenario0/forest_" + str(seed) + "_" + str(iters) + ".opt")
gbrt_loaded = load("optims/scenario0/gbrt_" + str(seed) + "_" + str(iters) + ".opt")